## Define function

In [1]:
import requests

# Base URL of the FastAPI application
BASE_URL = "http://localhost:8000"

# Example: Create a session
def create_session(lang, image, dockerfile, keep_template):
    url = f"{BASE_URL}/create_session/"
    payload = {
        "lang": lang,
        "image": image,
        "dockerfile": dockerfile,
        "keep_template": keep_template
    }
    response = requests.post(url, json=payload)
    return response.json()

# Example: Get active sessions
def get_sessions():
    url = f"{BASE_URL}/sessions/"
    response = requests.get(url)
    return response.json()
# Example: Get session metadata by session ID
def get_session_metadata(session_id):
    url = f"{BASE_URL}/session_metadata/{session_id}/"
    response = requests.get(url)
    return response.json()

# Example: Run code in a session
def run_code(session_id, code, libraries=None):
    url = f"{BASE_URL}/run_code/{session_id}/"
    payload = {
        "code": code,
        "libraries": libraries or []
    }
    response = requests.post(url, json=payload)
    return response.json()

# Example: Close a session
def close_session(session_id):
    url = f"{BASE_URL}/close_session/{session_id}/"
    response = requests.post(url)
    return response.json()

# Test CopyFromRuntimeRequest endpoint
def copy_from_runtime(session_id, src_runtime_file, dest_local_path):
    payload = {
        "src_runtime_file": src_runtime_file,
        "dest_local_path": dest_local_path
    }
    response = requests.post(f"{BASE_URL}/copy_from_runtime/{session_id}", json=payload)
    return response.json()

# Test CopyToRuntimeRequest endpoint
def copy_to_runtime(session_id, file_path, dest_runtime_path="./app"):
    """
    Test the copy_to_runtime endpoint.

    :param api_url: The base URL of the API (e.g., "http://localhost:8000")
    :param session_id: The ID of the session to copy the file to
    :param file_path: The local path of the file to be uploaded
    :param dest_runtime_path: The destination path in the runtime
    :return: The response from the API
    """
    with open(file_path, 'rb') as file:
        files = {'src': file}  # The file to upload
        data = {'dest_runtime_path': dest_runtime_path}  # The destination path in the runtime

        response = requests.post(f"{BASE_URL}/copy_to_runtime/{session_id}/", files=files, data=data)

    return response.json()

def add_mount(session_id: str, local_path: str, runtime_path: str, mount_type: str = "bind"):
    # Define the URL of your FastAPI application
    url = f"http://localhost:8000/add_mount/{session_id}/"

    # Define the payload for the request
    payload = {
        "local_path": local_path,
        "target": runtime_path,
        "type": mount_type
    }

    # Send the POST request
    response = requests.post(url, json=payload)
    return response


## create sessions (sandbox)

In [11]:
response = create_session(lang="python", image="",  dockerfile = "./docker_sandbox/Dockerfile",  keep_template= True)
print(response)
session_id=response['session_id']

{'message': 'Session created successfully.', 'session_id': '5978021c-f71c-47ca-9947-c27ec60b2391'}


## get all sessions

In [12]:
get_sessions()

{'active_sessions': ['2f1aae50-81d9-4b61-b2cc-8942fda837ec',
  '5978021c-f71c-47ca-9947-c27ec60b2391']}

## get session by id

In [64]:
response = get_session_metadata(session_id=session_id)
print(response)


{'session_id': '63a7c931-5f7c-4be9-b231-f474dd13ca5a', 'metadata': {'lang': 'python', 'dockerfile': './docker_sandbox/Dockerfile', 'keep_template': True}}


## run code

In [13]:
code = """
import numpy as np
print(np.pi)
"""
response = run_code(session_id=session_id, code=code, libraries="")
print(response)

{'output': '3.141592653589793\r\n'}


## close sessions

In [57]:
response = close_session(session_id)
response


{'message': 'Session closed successfully.'}

## copy from run time

In [14]:
code = """
import numpy as np
import matplotlib.pyplot as plt

# Generate x values
x = np.linspace(0, 2 * np.pi, 100)
# Calculate sine values
y = np.sin(x)

# Create the plot
plt.plot(x, y)
plt.title('Sine Wave')
plt.xlabel('x values (radians)')
plt.ylabel('sin(x)')
plt.grid()

# Save the figure
plt.savefig('./wave.png')
plt.show()
"""
response = run_code(session_id=session_id, code=code, libraries="")

# response = copy_from_runtime(session_id=session_id, src_runtime_file="wave.png", dest_local_path="mnt")
response

{'output': ''}

In [15]:
session_id

'5978021c-f71c-47ca-9947-c27ec60b2391'

## copy to run time

In [71]:
response = copy_to_runtime(session_id=session_id, file_path="requirements.txt")

## mount


In [9]:
response = add_mount(session_id=session_id, local_path="./mnt", runtime_path="./tmp")
response

<Response [422]>